In [1]:
from google.cloud import documentai_v1 as documentai
from google.oauth2 import service_account
from database import Database

db = Database()

# Base processor data
project_id = "genealogy-ocr-index"
location = "us"
processor_id = "4cb2ae40b145c48"

class ocr_engine:
    def __init__(self) -> None:

        # Create a client
        cred = service_account.Credentials.from_service_account_file("secrets/docai_credentials.json")
        self.__client = documentai.DocumentProcessorServiceClient(
            credentials=cred,
            client_options={"api_endpoint": f"{location}-documentai.googleapis.com"}
        )

        # Get the processor
        self.__processor = self.__client.processor_path(project=project_id, location=location, processor=processor_id)

    def process_documents(self):

        # Get image from URL
        uris = db.storage_get_images()

        # Get extension of images to set correct mime type
        extension = uris[2][-3:]
        if "jpg" in extension:
            extension = "jpeg"
        elif "png" in extension:
            extension = "png"

        # Create a raw document object
        document = documentai.GcsDocument(gcs_uri=uris[2], mime_type=f"image/{extension}")

        # Create API request
        request = documentai.ProcessRequest(name=self.__processor, 
                                            gcs_document=document,
                                            process_options={"ocr_config": {
                                                "hints": {"language_hints": "es"}
                                                }
                                            })

        # Get result
        result = self.__client.process_document(request=request)
        return result.document

App initialized


In [2]:
# Send API call and get result
engine = ocr_engine()
result = engine.process_documents()
text = result.text

In [3]:
# Load pre-trained and custom NER models for extraction
import spacy
from spacy.matcher import Matcher
nlp_es = spacy.load("es_core_news_lg")
nlp_custom = spacy.load("ner_model/model/model-best")

# Create a matcher to improve accuracy 
matcher = Matcher(nlp_es.vocab)

# Load list of names
import csv
names = []
last_names = []

with open('ner_model/data/processed/first_names_processed.csv', 'r', newline="") as file:
    reader = csv.reader(file, delimiter=",")
    for row in reader:
        names.append(row[0])

with open('ner_model/data/processed/last_names_processed.csv', 'r', newline='') as file:
    reader = csv.reader(file, delimiter=",")
    for row in reader:
        last_names.append(row[0])

# Add names and last names to entity ruler
ruler = nlp_custom.add_pipe("entity_ruler", after="ner", config={"overwrite_ents":True})

# Create patterns
patterns = []
match_patterns = []
for name in names:
    patterns.append({
        "label":"FIRST_NAME",
        "pattern": [{"LOWER":name.lower()}]
    })
    match_patterns.append([{"LOWER":name.lower()}])
    
# Add patterns to matcher
matcher.add("FIRST_NAME", match_patterns)

match_patterns = []
for name in last_names:
    patterns.append({
        "label":"LAST_NAME",
        "pattern": [{"LOWER":name.lower()}]
    })
    match_patterns.append([{"LOWER":name.lower()}])

# Add patterns to pipeline and matcher
matcher.add("LAST_NAME", match_patterns)
ruler.add_patterns(patterns)

In [ ]:
# Iterate over blocks to gather starting and ending indexes
import unicodedata
def get_indexes(doc):
    block_idx = []
    
    for page in doc.pages:
        for block in page.blocks:
            start = block.layout.text_anchor.text_segments[0].start_index
            end = block.layout.text_anchor.text_segments[0].end_index
            block_idx.append((start, end))

    return block_idx

# Get blocks of text for processing
def get_text(idx, text):
    start = idx[0]
    end = idx[1]

    return text[start:end]

# Process text to remove new line charachters
def formatting(text_block):
    document_text = []
    i = 0
    while i < len(text_block):
        if text_block[i] == "-" and text_block[i+1] == "\n":
            document_text.append("")
            i += 2
        elif text_block[i] == "\n" and text_block[i-1] != "":
            document_text.append(" ")
            i += 1
        elif text_block[i] != "ñ" and text_block[i] != "Ñ" and not text_block[i].isascii():
            normal_char = unicodedata.normalize("NFD", text_block[i])
            for c in normal_char:
                if unicodedata.category(c) != "Mn":
                    document_text.append(c)            
            i += 1
        else:
            document_text.append(text_block[i])
            i += 1

    #normalized_text = unicodedata.normalize("NFD", "".join(document_text))

    #return "".join(c for c in normalized_text if unicodedata.category(c) != 'Mn' and c != "ñ" and c != "Ñ")
    return "".join(document_text)

In [5]:
# Get blocks
block_idx = get_indexes(result)

# Get processed blocks of text
text_blocks = []
for idx in block_idx:
    text_block = get_text(idx, text)
    #print(f"Block: {text_block}")
    text_blocks.append(formatting(text_block))

print("Blocks: ", text_blocks)

Blocks:  ['648 ', '-62en el citio de la Loma donde se divide para Chiscote, la que hubo en el reparto del resguardo de Guacamayas.', '334)--Esc .292-- El Cocuy 25 de julio de 1858- Eleuterio y Julian Patricio venden a Joaquin 1 aria Espinel el derecho y accion que a cada uno le corresponde en las tierras del Alizal, ycortade ra vereda do. laa Tapias jurisdiccion de Chiscas, las que hubieron en el reparto del resguardo de indijenas de dicho Chiscas.------355)--Esc #293-- El Cocuy 25 de julio de 1858 Juan Crisostomo Ro driguez, vendor a Gavino Largo un pedazo de tierra en el sitio de Bocachica jurisdiccion de la Capilla; el que hubo por compra ', 'Π ', 'Javier Solano y Gregorio Murillo ', 'o a su pr ', '536)--Eso . 294-- ElCocuy 25 de julio de 1858-- Con este Numero y en la fecha queda protocolado el documento que Antonio Jose de Herrera otorgo a Mateo Maron y Francisco Javier Leal vendiendoles una estancia en el paramo del Contento jurisdiccion de Chiscas y Guicon ', '357)--Folio 512-- 

In [ ]:
# Extract names
names = []

## TODO: Need to find a way to be a bit more flexible with the choice 
for block in text_blocks:
    print("---------- New Block ----------")
    doc = nlp_es(block)
    for ent in doc.ents:
        if ent.label_ == "PER":
            print("## Default model results: ", ent, ent.label_)
            tokens = ent.text.split(" ")

            # Gather first names and last names
            first_names = []
            last_names = []

            # Identify names in the entity
            for token in tokens:
                name = nlp_custom(token)
                matches = matcher(name)
                for name_ent in name.ents:
                    if len(matches) > 0 and name_ent.label_ == nlp_es.vocab[matches[0][0]].text:
                        # Add to corresponding name list
                        if name_ent.label_ == "FIRST_NAME":
                            first_names.append(name_ent.text)
                        elif name_ent.label_ == "LAST_NAME":
                            last_names.append(name_ent.text)
                        
                        # Print recognition data
                        print("     -- MATCHES MATCHER. Custom model results: ", 
                              name_ent, 
                              name_ent.label_, 
                              "/ Matcher results: ", 
                              name[matches[0][1]:matches[0][2]],
                              nlp_es.vocab[matches[0][0]].text)
                        
                    elif len(matches) == 0:
                        continue

                    else:
                        print("     -- DISCREPANCY WITH MATCHER. Custom model results: ", 
                              name_ent, name_ent.label_, 
                              name[matches[0][1]:matches[0][2]], 
                              nlp_es.vocab[matches[0][0]].text)
            
            # Process names separated by a space
            for i in range(len(tokens)):
                if i+1 < len(tokens):
                    combined = nlp_custom(tokens[i]+tokens[i+1])
                    match = matcher(combined)
                    if match and combined.ents[0].label_ == nlp_es.vocab[match[0][0]].text:
                        # Add to corresponding name list
                        if combined.ents[0].label_ == "FIRST_NAME":
                            first_names.append(combined.ents[0].text)
                        elif combined.ents[0].label_ == "LAST_NAME":
                            last_names.append(combined.ents[0].text)
            
            # Print first and last names
            if first_names:
                print("     -- First Names: ", " ".join(first_names))
            if last_names:
                print("     -- Last Names: ", " ".join(last_names))
                


---------- New Block ----------
---------- New Block ----------
## Default model results:  Guacamayas PER
---------- New Block ----------
## Default model results:  Eleuterio PER
     -- MATCHES MATCHER. Custom model results:  Eleuterio FIRST_NAME / Matcher results:  Eleuterio FIRST_NAME
     -- First Names:  Eleuterio
## Default model results:  Julian Patricio PER
     -- MATCHES MATCHER. Custom model results:  Julian FIRST_NAME / Matcher results:  Julian FIRST_NAME
     -- MATCHES MATCHER. Custom model results:  Patricio FIRST_NAME / Matcher results:  Patricio FIRST_NAME
     -- First Names:  Julian Patricio
## Default model results:  Joaquin PER
     -- MATCHES MATCHER. Custom model results:  Joaquin FIRST_NAME / Matcher results:  Joaquin FIRST_NAME
     -- First Names:  Joaquin
## Default model results:  Alizal PER
## Default model results:  ycortade ra vereda do PER
     -- MATCHES MATCHER. Custom model results:  vereda LAST_NAME / Matcher results:  vereda LAST_NAME
     -- Last N